In [37]:
import pandas as pd
import math


In [2]:
def replace_space_in_col_names(df):
    new_cols = [x.replace(" ","_") for x in df.columns]
    df.columns = new_cols

In [175]:
definitive_df = pd.read_excel("../../data/DefinitiveHealthData.xlsx",0)
defintive_Quva_mappings_df = pd.read_excel("../../data/DefinitiveHealthData.xlsx",1)
trans_df = pd.read_excel("../../data/QuVa-GAIL DataRequest_4_5.xlsx",1)
replace_space_in_col_names(definitive_df)
replace_space_in_col_names(defintive_Quva_mappings_df)
replace_space_in_col_names(trans_df)

trans_df["trans_id"] = trans_df.apply(lambda x : str(x["Customer_ID"]) + "_" + str(x["Report_Date"]),axis=1)




definitive_df["Definitive_ID"] = definitive_df["Definitive_ID"].astype(str)
definitive_df["Definitive_IDN_ID"] = definitive_df["Definitive_IDN_ID"].astype(str)
definitive_df["Definitive_IDN_Parent_ID"] = definitive_df["Definitive_IDN_Parent_ID"].astype(str)


definitive_df["Definitive_ID"] = definitive_df["Definitive_ID"].astype(str)
definitive_df["Definitive_ID"] = definitive_df["Definitive_ID"].apply(lambda x : -1 if (str(x).strip() == "") or math.isnan(float(x)) else int(float(x)))
definitive_df["Definitive_ID"] = definitive_df["Definitive_ID"].astype(str) 

definitive_df["Definitive_IDN_ID"] = definitive_df["Definitive_IDN_ID"].astype(str)
definitive_df["Definitive_IDN_ID"] = definitive_df["Definitive_IDN_ID"].apply(lambda x : -1 if (str(x).strip() == "") or math.isnan(float(x)) else int(float(x)))
definitive_df["Definitive_IDN_ID"] = definitive_df["Definitive_IDN_ID"].astype(str) 

definitive_df["Definitive_IDN_Parent_ID"] = definitive_df["Definitive_IDN_Parent_ID"].astype(str)
definitive_df["Definitive_IDN_Parent_ID"] = definitive_df["Definitive_IDN_Parent_ID"].apply(lambda x : -1 if (str(x).strip() == "") or math.isnan(float(x)) else int(float(x)))
definitive_df["Definitive_IDN_Parent_ID"] = definitive_df["Definitive_IDN_Parent_ID"].astype(str) 


defintive_Quva_mappings_df["FinalDefID"] = defintive_Quva_mappings_df["FinalDefID"].astype(str)
defintive_Quva_mappings_df["FinalDefID"] = defintive_Quva_mappings_df["FinalDefID"].apply(lambda x : -1 if (str(x).strip() == "") or math.isnan(float(x)) else int(float(x)))
defintive_Quva_mappings_df = defintive_Quva_mappings_df[defintive_Quva_mappings_df.FinalDefID !=-1]
defintive_Quva_mappings_df["FinalDefID"] = defintive_Quva_mappings_df["FinalDefID"].astype(str) 





defintive_Quva_mappings_df["Customer_ID"] = defintive_Quva_mappings_df["Customer_ID"].astype(str)
trans_df["Customer_ID"] = trans_df["Customer_ID"].astype(str)


In [176]:
trans_def_joined_df = pd.merge(trans_df,defintive_Quva_mappings_df)
trans_def_joined_df = pd.merge(trans_def_joined_df,definitive_df,left_on="FinalDefID",right_on="Definitive_ID")

In [177]:
idn_parent_list = list(trans_def_joined_df["Definitive_IDN_ID"].unique())
idn_list = list(trans_def_joined_df["Definitive_IDN_Parent_ID"].unique())

In [178]:
trans_def_joined_df["is_idn_parent"] = trans_def_joined_df["Definitive_ID"].isin(idn_parent_list)
trans_def_joined_df["is_idn"] = trans_def_joined_df["Definitive_ID"].isin(idn_list)


tmplist = definitive_df[(definitive_df["Definitive_IDN_ID"] == "-1") & (definitive_df["Definitive_IDN_Parent_ID"] == "-1")]["Definitive_ID"].unique()
tmpcustlist = list(trans_def_joined_df[trans_def_joined_df.Definitive_ID.isin(tmplist)]["Customer_ID"].unique())

trans_def_joined_df["is_independent"] = trans_def_joined_df["Customer_ID"].apply(lambda x : True if x in tmpcustlist else False)

trans_def_joined_df["is_child"] = trans_def_joined_df.apply(lambda x : True if x["is_independent"] ==False and x["is_idn"] == False and x["is_idn_parent"] == False  else False,axis=1)

In [179]:
trans_def_joined_df.groupby(["is_idn_parent","is_idn","is_child","is_independent"])\
.agg({'Daily_Revenue':'sum',
     'trans_id':'nunique',
     }).reset_index()
#trans_def_joined_df[trans_def_joined_df.Customer_ID == "138"]
#trans_df[trans_df.Customer_ID == "138"]

,is_idn_parent,is_idn,is_child,is_independent,Daily_Revenue,trans_id
0,False,False,False,True,4.207268e+06,1515
1,False,False,True,False,5.250078e+07,13333
2,True,False,False,False,7.867784e+05,144
3,True,True,False,False,1.615061e+06,94


In [ ]:
trans_def_joined_df[(trans_def_joined_df["is_idn_parent"] == True) & (trans_def_joined_df["is_idn"] == True)].head().T

In [163]:
trans_def_joined_df[["is_child","Definitive_IDN_ID","Definitive_IDN_Parent_ID"]].sort_values(by='is_child',ascending=False).head()

,is_child,Definitive_IDN_ID,Definitive_IDN_Parent_ID
0,True,841666,-1
36490,True,4722,4722
36480,True,4722,4722
36481,True,4722,4722
36482,True,4722,4722


In [167]:
trans_def_joined_df[(trans_def_joined_df.is_idn_parent == True) & (trans_def_joined_df.is_idn == True)]["Definitive_ID"].unique()

array(['7235', '542032'], dtype=object)